In [103]:
'''
Created on Mar 24, 2011
Ch 11 code
@author: Peter
'''
from numpy import *

def loadData():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only gsets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #print "***",H1,"***"
            #print "$$$",freqSet,"$$$"
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        #print "conseq", conseq
        #print "freqSet", freqSet
        #print "freqSet-conseq", freqSet-conseq
        #print "supportData[freqSet]", supportData[freqSet]
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print freqSet-conseq,'-->',conseq,'conf:',conf
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    #print "###",H
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to  merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            
def pntRules(ruleList, itemMeaning):
    for ruleTup in ruleList:
        for item in ruleTup[0]:
            print itemMeaning[item]
        print "           -------->"
        for item in ruleTup[1]:
            print itemMeaning[item]
        print "confidence: %f" % ruleTup[2]
        print       #print a blank line
    

In [104]:
X = loadData()
L, supportData = apriori(X, 0.5)
print L
R = generateRules(L, supportData, 0.5)
print R



[[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])], [frozenset([1, 3]), frozenset([2, 5]), frozenset([2, 3]), frozenset([3, 5])], [frozenset([2, 3, 5])], []]
frozenset([3]) --> frozenset([1]) conf: 0.666666666667
frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0
frozenset([3]) --> frozenset([2]) conf: 0.666666666667
frozenset([2]) --> frozenset([3]) conf: 0.666666666667
frozenset([5]) --> frozenset([3]) conf: 0.666666666667
frozenset([3]) --> frozenset([5]) conf: 0.666666666667
frozenset([5]) --> frozenset([2, 3]) conf: 0.666666666667
frozenset([3]) --> frozenset([2, 5]) conf: 0.666666666667
frozenset([2]) --> frozenset([3, 5]) conf: 0.666666666667
[(frozenset([3]), frozenset([1]), 0.6666666666666666), (frozenset([1]), frozenset([3]), 1.0), (frozenset([5]), frozenset([2]), 1.0), (frozenset([2]), frozenset([5]), 1.0), (frozenset([3]), frozenset([2]), 0.6666666666666666), (frozenset([2]), fro

In [105]:
from numpy import *
def loadData():
    data = array([[1, 3, 4],
                 [2, 3, 5],
                 [1, 2, 3, 5],
                 [2, 5]])
    return data

def createC1(X):
    C1 = []
    for x in X:
        for a in x:
            if not [a] in C1:
                C1.append([a])
    C1.sort()
    C1 = map(frozenset, C1)
    return C1
    
def createCK(LK, k):
    CK =[]
    for i in xrange(len(LK)):
        for j in xrange(i+1, len(LK)):
            L1 = list(LK[i])[:k-2]
            L2 = list(LK[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                CK.append(LK[i] | LK[j])
    return CK    

def createLK(X, CK, minS):
    dictt = {}
    supportData = {}
    LK = []
    for x1 in X:
        for x2 in CK:
            if x2.issubset(x1):
                dictt[x2] = dictt.get(x2,0) + 1
    for key in dictt:
        s = 1.0 * dictt[key] / len(X)
        supportData[key] = s
        if s >= minS:
            LK.append(key)
    return LK, supportData
        
def apri(X, minS=0.5):
    C1 = createC1(X)
    X = map(set, X)
    L1, supportData = createLK(X, C1, minS)
    L = [L1]
    k = 2
    while len(L[k-2]) > 0:
        LK = L[k-2]
        CK = createCK(LK, k)
        LK, supK = createLK(X, CK, minS)
        supportData.update(supK)
        L.append(LK)
        k += 1
    return L, supportData


def gR(L, supportData, minC=0.7):
    R = []
    print L
    for i, X in enumerate(L[1:]):
        for x in X:
            H = [frozenset([item]) for item in x]
            print H
            if i > 1:
                rfc(x, H, supportData, R, minC)
            else:
                cc(x, H, supportData, R, minC)
    return R

def cc(x, H, supportData, R, minC=0.7):
    HH = []
    for item in H:
        conf = supportData[x] / supportData[x-item]
        if conf >= minC:
            R.append((x-item, item, conf))
            HH.append(item)
    return HH
            
def rfc(x, H, supportData, R, minC=0.7):
    m = len(H[0])
    if len(x) > m+1:
        HP = createCK(H, m+1)
        HP = cc(x, HP, supportData, R, minC)
        if len(HP) > 1:
            rfc(x, H, supportData, R, minC)   

In [106]:
X =loadData()
C1 = createC1(X)
X = map(set, X)
L1, supportData = createLK(X, C1, 0.5)
print L1
print supportData

[frozenset([5]), frozenset([2]), frozenset([3]), frozenset([1])]
{frozenset([4]): 0.25, frozenset([5]): 0.75, frozenset([2]): 0.75, frozenset([3]): 0.75, frozenset([1]): 0.5}


In [107]:
X = loadData()
L, supportData = apri(X, 0.5)
print L
print supportData
R = gR(L, supportData, 0.5)
print R

[[frozenset([5]), frozenset([2]), frozenset([3]), frozenset([1])], [frozenset([3, 5]), frozenset([2, 3]), frozenset([2, 5]), frozenset([1, 3])], [frozenset([2, 3, 5])], []]
{frozenset([5]): 0.75, frozenset([3]): 0.75, frozenset([2, 3, 5]): 0.5, frozenset([1, 2]): 0.25, frozenset([1, 5]): 0.25, frozenset([3, 5]): 0.5, frozenset([4]): 0.25, frozenset([2, 3]): 0.5, frozenset([2, 5]): 0.75, frozenset([1]): 0.5, frozenset([1, 3]): 0.5, frozenset([2]): 0.75}
[[frozenset([5]), frozenset([2]), frozenset([3]), frozenset([1])], [frozenset([3, 5]), frozenset([2, 3]), frozenset([2, 5]), frozenset([1, 3])], [frozenset([2, 3, 5])], []]
[frozenset([3]), frozenset([5])]
[frozenset([2]), frozenset([3])]
[frozenset([2]), frozenset([5])]
[frozenset([1]), frozenset([3])]
[frozenset([2]), frozenset([3]), frozenset([5])]
[(frozenset([5]), frozenset([3]), 0.6666666666666666), (frozenset([3]), frozenset([5]), 0.6666666666666666), (frozenset([3]), frozenset([2]), 0.6666666666666666), (frozenset([2]), frozenset